In [21]:
import optuna
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score


In [28]:
df = pd.read_csv('df_att.csv')
df.head()

X = df.drop('Heart Disease', axis=1)
y = df['Heart Disease']

cat_features = list(X.select_dtypes(exclude='number').columns)


skf = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)


def objective(trial):
    params = {
    "iterations": trial.suggest_int("iterations", 300, 1500),
    "depth": trial.suggest_int('depth', 4, 10),
    "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
    "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-3, 10, log=True),
    "border_count": trial.suggest_int("border_count", 32, 255),
    "random_state": 42,
    "cat_features": cat_features,
    "verbose": 0
    }
    
    scores = []
    
    for train_idx, valid_idx in skf.split(X, y):
    
        X_train_fold = X.iloc[train_idx]
        X_valid_fold = X.iloc[valid_idx]
        y_train_fold = y.iloc[train_idx]
        y_valid_fold = y.iloc[valid_idx]
        
        model = CatBoostClassifier(**params)

        model.fit(X_train_fold,
                  y_train_fold,
                  eval_set=(X_valid_fold, y_valid_fold),
                  use_best_model=False)
        
        probs = model.predict_proba(X_valid_fold)[:,1]

        score = roc_auc_score(y_valid_fold, probs)

        scores.append(score)
                  
    return np.mean(scores)



In [30]:
study = optuna.create_study(direction='maximize')

study.optimize(objective, n_trials=30, show_progress_bar=True)

[I 2026-02-17 10:47:21,705] A new study created in memory with name: no-name-61b47684-4630-4f94-92a9-3ef88d2ca5cc
Best trial: 0. Best value: 0.954431:   3%|▎         | 1/30 [09:35<4:38:07, 575.41s/it]

[I 2026-02-17 10:56:57,118] Trial 0 finished with value: 0.9544307359421605 and parameters: {'iterations': 441, 'depth': 7, 'learning_rate': 0.023075476101409025, 'l2_leaf_reg': 0.012660019013583133, 'border_count': 49}. Best is trial 0 with value: 0.9544307359421605.


Best trial: 1. Best value: 0.954681:   7%|▋         | 2/30 [23:54<5:46:17, 742.05s/it]

[I 2026-02-17 11:11:15,819] Trial 1 finished with value: 0.9546806704133578 and parameters: {'iterations': 427, 'depth': 10, 'learning_rate': 0.09828262971980745, 'l2_leaf_reg': 2.302887091547248, 'border_count': 146}. Best is trial 1 with value: 0.9546806704133578.


Best trial: 2. Best value: 0.955351:  10%|█         | 3/30 [40:29<6:25:55, 857.61s/it]

...

[I 2026-02-17 20:00:19,976] Trial 27 finished with value: 0.955448777300964 and parameters: {'iterations': 1082, 'depth': 6, 'learning_rate': 0.06098318978340682, 'l2_leaf_reg': 0.9712030715169957, 'border_count': 159}. Best is trial 25 with value: 0.9554952344394854.


Best trial: 25. Best value: 0.955495:  97%|█████████▋| 29/30 [9:35:14<21:57, 1317.10s/it]

[I 2026-02-17 20:22:35,891] Trial 28 finished with value: 0.9554660586253905 and parameters: {'iterations': 1290, 'depth': 5, 'learning_rate': 0.040702525672882336, 'l2_leaf_reg': 0.29951267009249116, 'border_count': 178}. Best is trial 25 with value: 0.9554952344394854.


Best trial: 25. Best value: 0.955495: 100%|██████████| 30/30 [9:56:35<00:00, 1193.18s/it]

[I 2026-02-17 20:43:57,133] Trial 29 finished with value: 0.9551822455028642 and parameters: {'iterations': 979, 'depth': 7, 'learning_rate': 0.02268785798960501, 'l2_leaf_reg': 4.0254686806132645, 'border_count': 149}. Best is trial 25 with value: 0.9554952344394854.
